In [1]:
import pandas as pd
import requests
import redcap
import io
import json

### import data into REDCap - QA environment

In [2]:
df_100 = pd.read_csv("100_records.csv")
df_1000 = pd.read_csv("1000_records.csv")
df_10000 = pd.read_csv("10000_records.csv")
df_100000 = pd.read_csv("100000_records.csv")

#making extra small dataframe
df_10 = df_100.loc[0:10,:]

In [3]:
qa_api_url = 'https://qa-redcap.doh.wa.gov/api/'
qa2_api_url = 'https://qa2-redcap.doh.wa.gov/api/'

qa_pid49 = "E3AA37D0ABCE88084F15E574432361A7"
qa2_pid49 = "EED20697ED77ADBEDC3E2392F802C23C"


#### import into QA - 10 records

In [4]:
project = redcap.Project(qa_api_url, qa_pid49)

In [5]:
%%time
response = project.import_records(df_10, force_auto_number=True)
response

Wall time: 7.49 s


{'count': 11}

#### import into QA - 100 records

In [ ]:
%%time
response = project.import_records(df_100, force_auto_number=True)
response

### import data into REDCap - QA2 environment

#### import into QA - 10 records

In [6]:
project = redcap.Project(qa2_api_url, qa2_pid49)

In [20]:
%%time
response = project.import_records(df_10, force_auto_number=True)
response

Wall time: 78.2 ms


{}

#### import into QA - 100 records

In [ ]:
%%time
response = project.import_records(df_100, force_auto_number=True)
response

#### import into QA - 1000 records

In [ ]:
%%time
response = project.import_records(df_1000, force_auto_number=True)
response

In [ ]:
%%time
response = project.import_records(df_10000, force_auto_number=True)
response

# Attempt to troubleshoot QA2 API import/export

### exporting data from QA2 (iaas) using native API
 - this works fine (as long as on VPN)

In [21]:
qa2_api_url = 'https://qa2-redcap.doh.wa.gov/api/'
qa2_pid49 = "EED20697ED77ADBEDC3E2392F802C23C"

#request form
fields = {
    'token': qa2_pid49,
    'content': 'record',
    'format': 'csv',
    'type': 'flat'}

#submit request
r = requests.post(qa2_api_url, data=fields)
print(r)

df = pd.read_csv(io.StringIO(r.content.decode("utf-8")), index_col=0)

print(df.shape)

<Response [200]>
(908, 138)


### exporting data from QA1 (paas) using native API
  - this works fine

In [22]:
qa2_api_url = 'https://qa-redcap.doh.wa.gov/api/'
qa2_pid49 = "EED20697ED77ADBEDC3E2392F802C23C"

#request form
fields = {
    'token': qa2_pid49,
    'content': 'record',
    'format': 'csv',
    'type': 'flat'}

#submit request
r = requests.post(qa2_api_url, data=fields)
print(r)

df = pd.read_csv(io.StringIO(r.content.decode("utf-8")), index_col=0)

print(df.shape)

<Response [403]>
(0, 0)


### import using pycap into QA2 (iaas)
 - this does NOT work!!!

In [ ]:
df_100 = pd.read_csv("100_records.csv")
df_10 = df_100.loc[0:10,:]

In [ ]:
qa2_api_url = 'https://qa2-redcap.doh.wa.gov/api/'
qa2_pid49 = "C601824C7A222588EC7F6D367BCF55C5"

import redcap
project = redcap.Project(qa2_api_url, qa2_pid49)

response = project.import_records(df_10, force_auto_number=True)
response

### export using pycap from QA2 (iaas)
 - this works just fine, WTF?!?!

In [ ]:
import redcap
project = redcap.Project(qa2_api_url, qa2_pid49)
x = project.export_records(format = "df")

print(x.shape)

### import using pycap into QA1 (paas)
- This works just fine, WTF?!?!

In [ ]:
qa_api_url = 'https://qa-redcap.doh.wa.gov/api/'
qa_pid49 = "C601824C7A222588EC7F6D367BCF55C5"

import redcap
project = redcap.Project(qa_api_url, qa_pid49)
response = project.import_records(df_10, force_auto_number=True)

response


### import using native API into QA1 (paas)


In [ ]:
record = df_10.loc[[0],:].to_csv()
print(record)

In [ ]:
# This finally worked

record_x = {
    'study_id': 1,
    'first_name': 'Boo',
    'last_name': 'Yaa'
}


data = json.dumps([record_x])

print(data)


qa_api_url = 'https://qa2-redcap.doh.wa.gov/api/'
qa_pid49 = "E3AA37D0ABCE88084F15E574432361A7"

fields = {
    'token': qa_pid49,
    'content': 'record',
    'format': 'json',
    'type': 'flat',
    'data': data,
    "forceAutoNumber": "true"
}

import requests
r = requests.post(qa_api_url, data=fields)
print('HTTP Status: ' + str(r.status_code))
print(r.text)

# Appendix

#### testing export json and csv formats

In [ ]:
qa2_api_url = 'https://qa-redcap.doh.wa.gov/api/'
qa2_pid49 = "E3AA37D0ABCE88084F15E574432361A7"

#request form
fields = {
    'token': qa2_pid49,
    'content': 'record',
    'format': 'json',
    'type': 'flat'}

#submit request
r = requests.post(qa2_api_url, data=fields)
print(r)

#df = pd.read_csv(io.StringIO(r.content.decode("utf-8")), index_col=0)

print(df.shape)